In [ ]:
import networkx as nx
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Nodes and Edges: How do we represent relationships between individuals using NetworkX?

As mentioned earlier, networks, also known as graphs, are comprised of individual entities and their representatives. The technical term for these are nodes and edges, and when we draw them we typically use circles (nodes) and lines (edges). 

In this notebook, we will work with a synthetic (i.e. simulated) social network, in which nodes are individual people, and edges represent their relationships. If two nodes have an edge between them, then those two individauls know one another. 

## Data Representation

In the `networkx` implementation, graph objects store their data in dictionaries. 

Nodes are part of the attribute `Graph.node`, which is a dictionary where the key is the node ID and the values are a dictionary of attributes. 

Edges are part of the attribute `Graph.edge`, which is a nested dictionary. Data are accessed as such: `G.edge[node1][node2]['attr_name']`.

Because of the dictionary implementation of the graph, any hashable object can be a node. This means strings and tuples, but not lists and sets.

# Synthetic Social Network

With this synthetic social network, we will attempt to answer the following basic questions using the NetworkX API:

1. How many people are present in the network?
2. What is the distribution of attributes of the people in this network?
2. How many relationships are represented in the network?
3. What is the distribution of the number of friends that each person has?

First off, let's load up the synthetic social network. This will show you through some of the basics of NetworkX.

For those who are interested, I simply created an Erdõs-Rényi graph with `n=30` and `p=0.1`. I used randomized functions that I wrote to generate attributes and append them to each node and edge. I then pickled the graph to disk.

In [ ]:
G = nx.read_gpickle('Synthetic Social Network.pkl')

# Basic Network Statistics
Let's first understand how many people and relationships are represented in the network.

In [ ]:
# Who are represented in the net work?
G.nodes() 

### Exercise

Can you write a single line of code that returns the number of individuals represented?

In [ ]:
___(G.nodes())

Let's now figure out who is connected to who in the network

In [ ]:
# Who is connected to who in the network?
G.edges()

### Exercise

Can you write a single line of code that returns the number of relationships represented?

In [ ]:
___(_._____())

## Concept

A **network**, more technically known as a **graph**, is comprised of:

- a set of nodes
- joined by a set of edges

They can be represented as two lists:

1. A **node list**: a list of 2-tuples where the first element of each tuple is the representation of the node, and the second element is a dictionary of metadata associated with the node.
2. An **edge list**: a list of 3-tuples where the first two elements are the nodes that are connected together, and the third element is a dictionary of metadata associated with the edge.

Since this is a social network of people, there'll be attributes for each individual, such as age, and sex. We can grab that data off from the attributes that are stored with each node.

In [ ]:
# Let's get a list of nodes with their attributes.
G.nodes(data=True)

# NetworkX will return a list of tuples in the form (node_id, attribute_dictionary) 

### Exercise

Can you count how many males and females are represented in the graph?

Hint: You may want to use the Counter object from the collections module.

In [ ]:
from collections import Counter
Counter([d['___'] for _, _ in _._____(data=True)])

Edges can also store attributes in their attribute dictionary.

In [ ]:
G.edges(data=True)

In this synthetic social network, I have stored the date as a datetime object. Datetime objects have attributes, namely `.year`, `.month`, `.day`.

### Exercise

Can you figure out the range of dates during which these relationships were forged? Specifically, compute the earliest and last date.

In [ ]:
# Answer
dates = [d['_____'] for _, _, d in G.edges(data=True)]
mindate = __________
maxdate = __________
print(mindate, maxdate)

### Exercise

We found out that there are two individuals that we left out of the network, individual no. 31 and 32. They are one male (31) and one female (32), their ages are 22 and 24 respectively, they knew each other on 2010-01-09, and together, they both knew individual 7, on 2009-12-11. Use the functions `G.add_node()` and `G.add_edge()` to introduce this data into the network.

If you need more help, check out https://networkx.github.io/documentation/latest/tutorial/tutorial.html

In [ ]:
# Answer
G.add_node(31, age=22, sex='Male')
# add one line here

G.add_edge(31, 32, date=datetime(2010,1,9))
# add two lines here 



G.node[31]

Verify that you have added in the edges and nodes correctly by running the following cell.

In [ ]:
def test_graph_integrity(G):
    assert 31 in G.nodes()
    assert 32 in G.nodes()
    assert G.has_edge(31, 32)
    assert G.has_edge(31, 7)
    assert G.has_edge(32, 7)
    print('All tests passed.')
    
test_graph_integrity(G)

## Tests

A note about the tests: Testing is good practice when writing code. Well-crafted assertion statements help you program defensivel, by forcing you to explicitly state your assumptions about the code. 

For more references on defensive programming, check out Software Carpentry's website: http://swcarpentry.github.io/python-novice-inflammation/08-defensive.html

For more information on writing tests for your data, check out these slides from a lightning talk I gave at Boston Python and SciPy 2015: http://j.mp/data-test

# Coding Patterns

These are some recommended coding patterns when doing network analysis using NetworkX, which stem from my roughly two years of experience with the package.

## Iterating using List Comprehensions
I would recommend that you use the following for compactness: 

    [d['attr'] for n, d in G.nodes(data=True)]

And if the node is unimportant, you can do:

    [d['attr'] for _, d in G.nodes(data=True)]

## Iterating over Edges using List Comprehensions

A similar pattern can be used for edges:

    [n2 for n1, n2, d in G.edges(data=True)]

or

    [n2 for _, n2, d in G.edges(data=True)]

If the graph you are constructing is a directed graph, with a "source" and "sink" available, then I would recommend the following pattern:

    [(sc, sk) for sc, sk, d in G.edges(data=True)]

or 

    [d['attr'] for sc, sk, d in G.edges(data=True)]

# Drawing Graphs

As illustrated above, we can draw graphs using the `nx.draw()` function. The most popular format for drawing graphs is the **node-link diagram**.

In [ ]:
nx.draw(G)

If the network is small enough to visualize, and the node labels are small enough to fit in a circle, then you can use the `with_labels=True` argument.

In [ ]:
nx.draw(G, with_labels=True)

However, note that if the number of nodes in the graph gets really large, node-link diagrams can begin to look like massive hairballs. This is undesirable for graph visualization.

Instead, we can use a **matrix** to represent them. The nodes are on the x- and y- axes, and a filled square represent an edge between the nodes. This is done by using the `nx.to_numpy_matrix(G)` function.

We then use `matplotlib`'s `pcolor(numpy_array)` function to plot. Because `pcolor` cannot take in numpy matrices, we will cast the matrix as an array of arrays, and then get `pcolor` to plot it.

In [ ]:
matrix = nx.to_numpy_matrix(G)

plt.pcolor(np.array(matrix))
plt.axes().set_aspect('equal') # set aspect ratio equal to get a square visualization
plt.xlim(min(G.nodes()), max(G.nodes())) # set x and y limits to the number of nodes present.
plt.ylim(min(G.nodes()), max(G.nodes()))
plt.title('Adjacency Matrix')
plt.show()

Let's try another visualization, the **Circos plot**. We can order the nodes in the Circos plot according to the node ID, but any other ordering is possible as well. Edges are drawn between two nodes.

Credit goes to Justin Zabilansky (MIT) for the implementation, and Jon Charest for improvements.

In [ ]:
from circos import CircosPlot

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)

nodes = sorted(G.nodes())
edges = G.edges()

c = CircosPlot(nodes, edges, radius=10, ax=ax)
c.draw()

It's pretty obvious in this visualization that there are nodes, such as node 5 and 18, that are not connected to any other node via an edge. There are other nodes, like node number 19, which is highly connected to other nodes.

Finally, let's try **hive plots** for the network. Two groups (male and female), and then edges drawn between them.

In [ ]:
from hiveplot import HivePlot

nodes = dict()
nodes['male'] = [n for n,d in G.nodes(data=True) if d['sex'] == 'Male']
nodes['female'] = [n for n,d in G.nodes(data=True) if d['sex'] == 'Female']

edges = dict()
edges['group1'] = G.edges(data=True)

nodes_cmap = dict()
nodes_cmap['male'] = 'blue'
nodes_cmap['female'] = 'red'

edges_cmap = dict()
edges_cmap['group1'] = 'black'

In [ ]:
h = HivePlot(nodes, edges, nodes_cmap, edges_cmap)
h.draw()